# Что нужно сделать 
1. Удалить structure_
2. Геометрические признаки ( сферичность, объём и площадь(в нм))
3. Total_magnetization по формуле (надо подумать как считать знаменатель) +/-


4. Pymatgen и ПРОБЛЕМЫ
[
- 1)  Массовое соотношение атомов + масса атомамная масса/общая масса формулы
2)  Number of magnetic ions per unit cell - что-то я не пойму как
3) get_exchange_group_info + уже изначало есть, так что использовать функцию не имеет смысла (но, из групп нельзя вывести lattice parameter и плотность, это надо будет делать отдельно заново)
4) number_of_unique_magnetic_sites - проблемнно, так как нет cif файлов
5) number_of_magnetic_sites - проблемнно, так как нет cif файлов
]

# Начало

In [19]:
import pandas as pd
import numpy as np
import re # Регулярки
from math import gcd, isclose
import sympy as sp # Библиотека символьных вычеслений, мат.задачи

In [20]:
from pymatgen.core.structure import Composition
from pymatgen.core import Structure, Element

In [21]:
df = pd.read_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data_1.csv')

In [22]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         533 non-null    int64  
 1   c1                 533 non-null    object 
 2   c2                 322 non-null    object 
 3   c3                 9 non-null      object 
 4   chemical_formula   533 non-null    object 
 5   crystal_structure  533 non-null    object 
 6   depth_nm           509 non-null    float64
 7   id                 533 non-null    int64  
 8   space_group_shell  533 non-null    int64  
 9   structure          533 non-null    object 
 10  structure_         533 non-null    object 
 11  formula_combined   533 non-null    object 
 12  coer_oe            533 non-null    float64
 13  exc_bias_oe        533 non-null    float64
 14  fc_field_t         533 non-null    float64
 15  h_range_max_koe    533 non-null    float64
 16  length_d_nm        533 non

# Исправим стотлбец formula_combined

Проблема была в том, что формулы, что должны были иметь вид Li2 Cr2 O4 имели вид Li2 Cr1 O3 Cr1 O1

In [23]:
print(df['formula_combined'].unique()) # Пример, что нас не устраивает

['Zn1 O1 Fe2 Co1 O4' 'Co1 O1 Fe2 Co1 O4' 'Mn1 Zn1 Fe4 O8' 'La1 Fe1 O3'
 'Ni1 Ni1 O1' 'Mg1 Fe2 O4' 'Fe2 Co1 O4' 'Fe1 Bi1 O3'
 'Fe2 Co1 O4 Fe1 Bi1 O3' 'Co1 Co1 O1' 'Fe1 Rh1' 'Mn1 O2' 'Ag1 Fe3 O4'
 'Fe1 Fe2 O3' 'Fe1 O1 Fe3 O4' 'Ni1 O1' 'Fe3 O4' 'Fe3 O4 Ni1 O1'
 'Ni1 O1 Fe3 O4' 'Fe4 Bi2 O9 Fe1 Bi1 O3' 'Fe1 Bi1 O3 Fe4 Bi2 O9'
 'Ti1 O2 Fe2 O3' 'Fe13 Co2 O20' 'Fe23 Co7 O40' 'Fe7 Co3' 'Fe1 Co1'
 'Fe3 Co7' 'Fe1 Fe3 O4' 'La2 Mn1 Fe1 O6' 'V1 Fe1 O4 Fe3 O4' 'Au1 Fe3 O4'
 'Gd1 Ti1 Fe9 Bi9 O30' 'Sr11 Nd39 Co50 Cu97 O150' 'Fe3 O4 Co1 O1'
 'Fe2 Co1 O4 Co1 O1' 'Cu98 Fe1 O50' 'Cu96 Fe1 O25' 'Fe2 O3'
 'Sr3 La17 Fe20 O60' 'Co1 Co3 O4' 'Mn3 O4' 'Sr33 La67 Mn100 O300'
 'Mn1 O1 Mn3 O4' 'Fe3 O4 Fe2 O3' 'Co3 O4' 'Fe1 Ni99 O100' 'Fe1 Ni49 O50'
 'Fe1 Ni24 O25' 'Fe3 Ni97 O100' 'Cr5 Ag1 O8' 'Cr1 Ag1 O2'
 'Co1 O1 Fe2 Ni1 O4' 'Mn1 Fe2 O4' 'Mn1 Fe2 O4 Fe3 O4' 'La1 Fe10 Bi9 O30'
 'Nd1 Fe1 O3' 'Fe2 Co1 O4 Fe3 O4' 'Dy1 Ti1 Cr1 O5' 'Gd1 Ti1 Cr1 O5'
 'Ni1 O1 Mn2 Co1 O4' 'Ba1 Fe12 O19 Fe3 O4' 'Fe2 Ni1 O4 Co1 O1'
 'Cr2 Co1 

Пример работы с такой ошибкой

1. Создание объекта Composition из вашей формулы
comp = Composition("Zn1 O1 Fe2 Co1 O4") # Может теперь можно склеить их обратно

2. Получение словаря количества атомов каждого элемента
el_amt_dict = comp.get_el_amt_dict()

3. Собираем обратно химическую формулу из словаря
formula = ""
for el, amt in el_amt_dict.items():
    formula += f"{el}{'' if amt == 1 else str(int(amt))}"

print(Composition(formula))

In [24]:
def transform_formula(formula):
    '''
    Zn1 O1 Fe2 Co1 O4 -> {'Zn': 1.0, 'O': 5.0, 'Fe': 2.0, 'Co': 1.0} -> Zn1 O5 Fe2 Co1
    '''
    comp = Composition(formula)
    el_amt_dict = comp.get_el_amt_dict() # Получение словаря с количеством атомов в формуле
    # Обратная сборка
    new_formula = ''
    for el, amt in el_amt_dict.items():
        new_formula += f"{el}{'' if amt == 1 else str(int(amt))}"
    return new_formula

In [25]:
def process_column(df, column_name):
    """
    Функция проверяет каждую ячейку в столбце DataFrame на равенство 0 или NaN.
    Если значение не равно 0 или NaN, оно заменяется на результат применения класса Composition из библиотеки pymatgen.

    Args:
        df (pandas.DataFrame): Исходный DataFrame.
        column_name (str): Имя столбца, который нужно обработать.

    Returns:
        pandas.DataFrame: DataFrame c обработанным столбцом.
    """
    new_column = []
    for value in df[column_name]:
        if pd.notna(value) and value != 0:
            # Если значение не NaN и не равно 0, применяем класс Composition
            comp = Composition(str(value))
            new_value = comp.formula
        else:
            new_value = value
        new_column.append(new_value)

    df[column_name] = new_column
    return df

In [26]:
df['formula_combined'] = df['formula_combined'].apply(transform_formula)
df = process_column(df, 'formula_combined')
print(df['formula_combined'].unique()) # Тепрь у нас есть полный средний материал ядра+оболочки

['Zn1 Fe2 Co1 O5' 'Fe2 Co2 O5' 'Mn1 Zn1 Fe4 O8' 'La1 Fe1 O3' 'Ni2 O1'
 'Mg1 Fe2 O4' 'Fe2 Co1 O4' 'Fe1 Bi1 O3' 'Fe3 Co1 Bi1 O7' 'Co2 O1'
 'Fe1 Rh1' 'Mn1 O2' 'Fe3 Ag1 O4' 'Fe3 O3' 'Fe4 O5' 'Ni1 O1' 'Fe3 O4'
 'Fe3 Ni1 O5' 'Fe5 Bi3 O12' 'Ti1 Fe2 O5' 'Fe13 Co2 O20' 'Fe23 Co7 O40'
 'Fe7 Co3' 'Fe1 Co1' 'Fe3 Co7' 'Fe4 O4' 'La2 Mn1 Fe1 O6' 'V1 Fe4 O8'
 'Fe3 Au1 O4' 'Gd1 Ti1 Fe9 Bi9 O30' 'Sr11 Nd39 Co50 Cu97 O150'
 'Fe3 Co1 O5' 'Fe1 Cu98 O50' 'Fe1 Cu96 O25' 'Fe2 O3' 'Sr3 La17 Fe20 O60'
 'Co4 O4' 'Mn3 O4' 'Sr33 La67 Mn100 O300' 'Mn4 O5' 'Fe5 O7' 'Co3 O4'
 'Fe1 Ni99 O100' 'Fe1 Ni49 O50' 'Fe1 Ni24 O25' 'Fe3 Ni97 O100'
 'Cr5 Ag1 O8' 'Cr1 Ag1 O2' 'Fe2 Co1 Ni1 O5' 'Mn1 Fe2 O4' 'Mn1 Fe5 O8'
 'La1 Fe10 Bi9 O30' 'Nd1 Fe1 O3' 'Fe5 Co1 O8' 'Dy1 Ti1 Cr1 O5'
 'Gd1 Ti1 Cr1 O5' 'Mn2 Co1 Ni1 O5' 'Ba1 Fe15 O23' 'Cr2 Co1 O4' 'Fe1 Ni1'
 'Mn4 O4' 'Mn1 Fe3 O5' 'Fe4 C1' 'Mg1 Fe3 O5' 'Mg1 Fe5 Co1 O9' 'Fe2 Co1 O3'
 'Fe4 Co1 Ni1 O8' 'Fe2 Ni1 O4' 'Mn1 Fe4 O7' 'Fe8 C2 O4' 'Fe5 C2' 'Fe1 Ni3'
 'Fe1 Ni3 Mo1' 'Fe11 C5' 'Fe11 

# Добавление дескрипторов 
## Pymatgen

// TODO: добавление параемтров обязательно 
1, 3 -  можно посчитать для всего материала ядро-оболочка в целом, наверное

1. Массовое соотношение атомов  +
2. Number of magnetic ions per unit cell
3. get_exchange_group_info + (так как уже руками собрали)
4. number_of_unique_magnetic_sites
5. number_of_magnetic_sites

from pymatgen.core.composition import Composition

compound_formula = "Fe2O3"  # пример химической формулы вещества
composition = Composition(compound_formula)
total_num_magnetic_ions = sum(composition.get_el_amt_dict().values())  # суммирование количества ионов для каждого элемента
print("Total number of magnetic ions per unit cell:", total_num_magnetic_ions)

In [27]:
# Пример для молярной массы всего вещества
from pyteomics import mass
print(mass.calculate_mass(formula='Fe3'))

167.8048125


In [28]:
def calculate_mass_ratios(formula):
    total_mass = mass.calculate_mass(formula=formula.replace(' ', ''))  # Вычисление общей массы соединения
    elements = {}  # Словарь для хранения массовых соотношений
    components = formula.split()  # Разделение формулы на компоненты
    for component in components:
        element_name = ''.join(filter(str.isalpha, component))  # Извлечение названия элемента
        atoms_count = int(''.join(filter(str.isdigit, component))) if component[-1].isdigit() else 1  # Количество атомов
        element_mass = mass.calculate_mass(formula=element_name)  # Масса элемента
        element_mass_ratio = element_mass * atoms_count / total_mass  # Расчет массового соотношения для элемента
        elements[element_name] = element_mass_ratio  # Запись массового соотношения для элемента
    return elements

In [29]:
df['mass_ratio'] = df['formula_combined'].apply(calculate_mass_ratios)
print(df['mass_ratio']) # Было
mass_ratio_values = [[value for value in dct.values()] for dct in df['mass_ratio']]
df['mass_ratio'] = mass_ratio_values 
print(df['mass_ratio']) # Стало

0      {'Zn': 0.20313874751542227, 'Fe': 0.3554733492...
1      {'Fe': 0.3612074917593743, 'Co': 0.38056914870...
2      {'Mn': 0.11674879913946458, 'Zn': 0.1358557729...
3      {'Mn': 0.11674879913946458, 'Zn': 0.1358557729...
4      {'Mn': 0.11674879913946458, 'Zn': 0.1358557729...
                             ...                        
528    {'La': 0.43852752646137455, 'Fe': 0.1765866657...
529    {'La': 0.43852752646137455, 'Fe': 0.1765866657...
530    {'La': 0.43852752646137455, 'Fe': 0.1765866657...
531    {'La': 0.43852752646137455, 'Fe': 0.1765866657...
532    {'La': 0.43852752646137455, 'Fe': 0.1765866657...
Name: mass_ratio, Length: 533, dtype: object
0      [0.20313874751542227, 0.355473349245203, 0.187...
1      [0.3612074917593743, 0.3805691487063179, 0.258...
2      [0.11674879913946458, 0.13585577295807472, 0.4...
3      [0.11674879913946458, 0.13585577295807472, 0.4...
4      [0.11674879913946458, 0.13585577295807472, 0.4...
                             ...           

3. Get_exchange_group_info из pymatgen - возвращает значения пространственную группу в виде число-буквы, но думаю, нам не нужно добирать эти дескрипторы, так как мы сделали это на этапе сбора датасета вручную 

In [30]:
print(df['c2'].unique(), df['c3'].unique(), sep='\n')

['Fe2 Co1 O4' nan 'Ni1 O1' 'Fe1 Bi1 O3' 'Co1 O1' 'Fe3 O4' 'Fe2 O3'
 'Fe4 Bi2 O9' 'Fe1 O50' 'Fe1 O25' 'Co3 O4' 'Mn3 O4' 'Fe2 Ni1 O4'
 'Mn2 Co1 O4' 'Mn1 O1' 'Fe3 C1' 'Mg1 O1' 'Mo1' 'Co1' 'Fe1']
[nan 'Fe2 Co1 O4']


# Геометрические признаки

Помним что все наши формы закодированы:
'''
{
    1: ['sphere', 'quasi-spherical', 'quasi-sphere', 'pseudo-spherical', 'pseudospherically', 'egg', 'flower'], 
    2: ['octahedral', 'octahedron', 'octahedral '],  
    3: ['cubic', 'cube'],
    4: ['hexahedron', 'hexagonal'],
    5: ['tetrahedral', 'triangle'],
    6: ['wire', 'rod'], 
    7: ['octopod']
}
'''

In [31]:
df_renamed = df.rename(columns={'width_nm': 'x', 'length_d_nm': 'y', 'depth_nm': 'z'})  # Для удобства теперь x, y, z
df = df.rename(columns={'width_nm': 'x', 'length_d_nm': 'y', 'depth_nm': 'z'})

In [32]:
print(df['shape'].unique())

[1. 5. 2. 6. 3. 7. 4.]


In [33]:
# Добавим колонки для наших значений 
df.insert(0, 'area', 0)
df.insert(0, 'volume', 0)
df = df.astype({'volume': float ,'area': float})

In [34]:
# Создадим словари для подсчёта площади и объёма
dict_area = {
    1: lambda x, y=1, z=1: 4 * 3.14159 * x ** 2, 
    2: lambda x, y=1, z=1: (3 ** 2) * 2 * x ** 2, 
    3: lambda x, y=1, z=1: 6 * x **2, 
    4: lambda x, y=1, z=1: 2 * x ** 2 + 4 * x * z/2,  
    5: lambda x, y=1, z=1: (3 ** (1/2)) * x ** 2, 
    6: lambda x, y, z=1: 2 * 3.14 * (x/2) * x +  2 * 3.14 * (x/2) * x ** 2, 
    7: lambda x, y=1, z=1: (2 ** (1/2)) * 2 * x ** 2, 
}

dict_volume = {
    1: lambda x, y=1, z=1: (4/3) * 3.14159 * x ** 3, 
    2: lambda x, y=1, z=1: (2/3) * (2 ** 2) * x ** 3, 
    3: lambda x, y=1, z=1: x ** 3, 
    4: lambda x, y=1, z =1: x ** 2 * x/2,  
    5: lambda x , y=1, z=1: (((2) ** (1/2)) * x ** 3) / 12,  
    6: lambda x, y, z=1: 3.14159 * (x ** 2) * (y / 2),
    7: lambda x, y=1, z=1: (2/3) * (2 ** (1/2)) * x ** 3,
}

In [35]:
# Создаем функцию для вычисления площади
def calculate_area(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)
    
    if shape in dict_area:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_area[shape](x, y) if shape in [3, 6] else dict_area[shape](x, z) if shape == 4 else dict_area[shape](x)
    else:
        return 'error'

# Создаем функцию для вычисления объема
def calculate_volume(row):
    shape = float(row['shape'])
    x = row['x']
    y = row.get('y', 0)
    z = row.get('z', 0)
    
    if shape in dict_volume:
        if shape in [1, 2, 3, 4, 5, 6, 7]:
            return dict_volume[shape](x, y, z) if shape in [3, 6] else dict_volume[shape](x, z) if shape == 4 else dict_volume[shape](x)
    else:
        return 'error'

In [36]:
df['area'] = df.apply(calculate_area, axis=1)
df['volume'] = df.apply(calculate_volume, axis=1)

In [37]:
print(df['area'].unique())

[6.88133874e+02 6.15751640e+02 3.63167804e+03 1.52052956e+03
 1.80955584e+03 4.07150064e+03 5.54176476e+03 9.16087644e+03
 1.62860026e+04 1.13097240e+04 2.49415316e+02 4.43405007e+02
 1.03680000e+04 1.96020000e+04 3.93691492e+03 5.22961638e+03
 7.42030992e+03 1.36019537e+04 7.85397500e+03 5.02654400e+05
 2.63760000e+04 5.30928710e+04 1.70553780e+03 3.76098588e+03
 4.52310900e+03 1.36847660e+04 3.40055805e+04 1.26676763e+04
 4.53645596e+03 2.12371484e+03 2.09116797e+03 2.82743100e+03
 3.54672945e+03 4.34745791e+03 5.02654400e+03 1.21500000e+04
 1.38240000e+04 1.56060000e+04 2.40000000e+03 3.21698816e+03
 1.76776695e+03 3.52800000e+03 6.14400000e+03 7.35000000e+03
 2.12176224e+03 2.39313760e+03 1.25663600e+03 8.04247040e+02
 5.83200000e+03 1.60459851e+03 4.02638741e+03 6.82215118e+03
 1.34354395e+04 8.15334068e+03 2.01061760e+04 2.43284730e+04
 3.01718304e+04 5.14718106e+04 1.80123641e+04 2.32351996e+04
 1.77952224e+03 1.38544119e+03 1.20687321e+03 9.95381376e+04
 1.01787516e+05 5.708595

In [38]:
def calculate_sphericity(row):
    area = row['area']
    volume = row['volume']   # Предполагается, что у нас есть столбцы 'area' и 'volume'
    radius_sphere = (3 * volume / (4 * np.pi)) ** (1/3) # Вычисляем радиус сферы по объему
    surface_area_sphere = 4 * np.pi * radius_sphere ** 2 # Вычисляем поверхность сферы
    sphericity = surface_area_sphere / area # Вычисляем сферичность
    return sphericity

In [39]:
df['sphericity'] = df.apply(calculate_sphericity, axis=1)

In [40]:
print(df['sphericity'].unique())

[1.00000028 1.00000028 1.00000028 0.67113929 0.67113929 0.51664327
 1.00000028 1.34501829 0.37248422 0.80599598 0.80599598 0.80599598
 1.64394881 0.67113929 0.51664327 0.09100827 0.0563519  0.51664327
 0.51664327 1.47410027 0.07037894 0.80599598]


Как можно заметить наш датасет имеет частицы сферические 1, и близкие к сферическим 0.8 и деформированные, совсем нее сферические 0.05 и 0.37, а значения 1.64394881 - наверное, указывают на не симметричные частицы

In [41]:
print(df['chemical_formula'].unique())

["['ZnO', 'CoFe2O4']" "['CoO', 'CoFe2O4']" "['Mn0.50Zn0.50Fe2O4']"
 "['LaFeO3']" "['Ni', 'NiO']" "['MgFe2O4']" "['CoFe2O4']" "['BiFeO3']"
 "['CoFe2O4', 'BiFeO3']" "['Co', 'CoO']" "['FeRh']" "['MnO2']"
 "['Ag', 'Fe3O4']" "['Fe', 'Fe2O3']" "['FeO', 'Fe3O4']" "['NiO']"
 "['Fe3O4']" "['Fe3O4', 'NiO']" "['NiO', 'Fe3O4']"
 "['Bi2Fe4O9', 'BiFeO3']" "['BiFeO3', 'Bi2Fe4O9']" "['TiO2', 'Fe2O3']"
 "['Co0.4Fe2.6O4']" "['Co0.7Fe2.3O4']" "['Fe0.7Co0.3']" "['Fe0.5Co0.5']"
 "['Fe0.3Co0.7']" "['Fe', 'Fe3O4']" "['La2FeMnO6']" "['FeVO4', 'Fe3O4']"
 "['Au', 'Fe3O4']" "['Bi0.9Gd0.1Fe0.9Ti0.1O3']" "['Nd0.78Sr0.22CoO3']"
 "['Fe3O4', 'CoO']" "['CoFe2O4', 'CoO']" "['Cu0.98', 'Fe0.02O']"
 "['Cu0.96', 'Fe0.04O']" "['Fe2O3']" "['La0.85Sr0.15FeO3']"
 "['Co', 'Co3O4']" "['Mn3O4']" "['La0.67Sr0.33MnO3']" "['MnO', 'Mn3O4']"
 "['Fe3O4', 'Fe2O3']" "['Co3O4']" "['Ni0.99Fe0.01O']" "['Ni0.98Fe0.02O']"
 "['Ni0.96Fe0.04O']" "['Ni0.97Fe0.03O']" "['Ag0.5Cr2.5O4']" "['AgCrO2']"
 "['CoO', 'NiFe2O4']" "['MnFe2O4']" "['MnFe2O4', 

# Значение целевой переменной и вертикального сдвига
Есть небольшая проблема. Часть данных отрицательная поскольку минус показывет направление, я хочу сделать направления отдельными колонками. Где 1 - число было отрицательным, 0 - положительным

# Last

In [42]:
print(df.drop(columns=['Unnamed: 0', 'structure_', 'structure']))

            volume          area  Unnamed: 0              c1          c2   c3  \
0      1697.396888    688.133874           0          Zn1 O1  Fe2 Co1 O4  NaN   
1      1436.753827    615.751640           1          Co1 O1  Fe2 Co1 O4  NaN   
2     20579.508893   3631.678040           2  Mn1 Zn1 Fe4 O8         NaN  NaN   
3      5575.275053   1520.529560           3  Mn1 Zn1 Fe4 O8         NaN  NaN   
4      7238.223360   1809.555840           4  Mn1 Zn1 Fe4 O8         NaN  NaN   
..             ...           ...         ...             ...         ...  ...   
528  268082.346667  20106.176000         528      La1 Fe1 O3      Ni1 O1  NaN   
529  268082.346667  20106.176000         529      La1 Fe1 O3      Ni1 O1  NaN   
530  179594.228333  15393.791000         530      La1 Fe1 O3      Ni1 O1  NaN   
531  179594.228333  15393.791000         531      La1 Fe1 O3      Ni1 O1  NaN   
532  179594.228333  15393.791000         532      La1 Fe1 O3      Ni1 O1  NaN   

          chemical_formula 

In [43]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 28 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   volume             533 non-null    float64
 1   area               533 non-null    float64
 2   Unnamed: 0         533 non-null    int64  
 3   c1                 533 non-null    object 
 4   c2                 322 non-null    object 
 5   c3                 9 non-null      object 
 6   chemical_formula   533 non-null    object 
 7   crystal_structure  533 non-null    object 
 8   z                  509 non-null    float64
 9   id                 533 non-null    int64  
 10  space_group_shell  533 non-null    int64  
 11  structure          533 non-null    object 
 12  structure_         533 non-null    object 
 13  formula_combined   533 non-null    object 
 14  coer_oe            533 non-null    float64
 15  exc_bias_oe        533 non-null    float64
 16  fc_field_t         533 non

In [44]:
df.to_csv(r'C:\Users\Ксения\Project\Magnet_Exchange_bias\data\Bias_data_2.csv')